# **Tuning Hyperparameters**

## Objectives

* We will tune the hyperparameters of for Logistic Regression, Ada Boost, and Gradient Boosting models.

## Inputs

* Training and Testing data sets from notebook 04.
* Insights developed in the previous notebook.

## Outputs

* We will have saved models with tuned hyper parameters at the end of this notebook.

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory
We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os

home_dir = '/workspace/pp5-ml-dashboard'
os.chdir(home_dir)
current_dir = os.getcwd()
print(current_dir)

/workspace/pp5-ml-dashboard


We now load our training and test sets, as well as some of the packages that we will be using.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from src.utils import get_df, save_df

train_dir = 'datasets/train/csv'
X_TrainSet = get_df('X_TrainSet',train_dir)
Y_TrainSet = get_df('Y_TrainSet',train_dir)

test_dir = 'datasets/test/csv'
X_TestSet = get_df('X_TestSet',test_dir)
Y_TestSet = get_df('Y_TestSet',test_dir)

## Section 1: Pipeline and Grid Search set up
We recall the code for building our pipelines and the grid search that we performed in the last notebook. Note that some of the constants have changed. 

We have modified the pipeline to see how feateure selection impacts the performance. Note that setting `thresh=1` essentially removes the `'corr_selector'` step from the pipeline. We will eventually remove this step from the pipeline once we have selected a value for `thresh`.

In [3]:
from sklearn.preprocessing import StandardScaler
from feature_engine import transformation as vt
from feature_engine.selection import DropFeatures, SmartCorrelatedSelection
from sklearn.pipeline import Pipeline


# Constants needed for feature engineering
TO_DROP = ['ftm_away', 'plus_minus_home', 'fg3m_away', 'pts_away', 'play_off',
           'fgm_away', 'pts_home', 'fg3m_home', 'ftm_home', 'fgm_home',
           'season']
THRESH = 0.6
TRANSFORMS = {'box_cox':(vt.BoxCoxTransformer,False),
              'yeo_johnson':(vt.YeoJohnsonTransformer,False)}
TRANSFORM_ASSIGNMENTS = {
    'yeo_johnson': ['dreb_away', 'blk_home', 'oreb_away', 'fta_away',
                    'dreb_home', 'ast_home', 'stl_away', 'stl_home',
                    'reb_away', 'oreb_home', 'pf_away', 'pf_home'],
    'box_cox': ['ast_away', 'fta_home']
                            }


def base_pipeline(thresh=THRESH):
    pipeline = Pipeline([
        ('dropper', DropFeatures(features_to_drop=TO_DROP)),
        ('corr_selector', SmartCorrelatedSelection(method="pearson",
                                                   threshold=thresh,
                                                   selection_method="variance")
                                                   )
                        ])
    return pipeline

    
def add_transformations(pipeline, transform_assignments):
    # This needs to be called after the above is fit so that the correlation selector has that attr
    dropping = pipeline['corr_selector'].features_to_drop_
    
    new_assignments = { key: [val for val in value if val not in dropping] 
                       for key,value in transform_assignments.items()}
    for transform, targets in new_assignments.items():
        if not targets:
            continue
        pipeline.steps.append(
            (transform, TRANSFORMS[transform][0](variables=targets))
            )
    pipeline.steps.append(('scaler', StandardScaler()))
    return pipeline

In [4]:
from sklearn.feature_selection import SelectFromModel

# ML algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier


MODELS = {
    'LogisticRegression': LogisticRegression,
    'GradientBoosting': GradientBoostingClassifier,
    'AdaBoost': AdaBoostClassifier,
}

def create_pipe(model_name, random_state=42, params={}):
    model = MODELS[model_name](random_state=random_state,**params)
    base_pipe = base_pipeline()
    base_pipe.fit(X_TrainSet)
    pipe= add_transformations(base_pipe,TRANSFORM_ASSIGNMENTS)
    pipe.steps.append(("feat_selection", SelectFromModel(model)))
    pipe.steps.append(('model',model))
    pipe.model_type = model_name
    pipe.name = model_name
    return pipe


Next, we have the code for our grid search. As we will be treating `thresh` as a hyperparameter, it will be slightly different.

In [5]:
from sklearn.model_selection import GridSearchCV
# to suppress warnings, I think
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import logging
logging.captureWarnings(True)
os.environ['PYTHONWARNINGS']='ignore'


def grid_search(X_train, y_train,pipe,param_grid={},verbosity=1):
    print(f"### Beginning grid search for {pipe.name} ###") 
    grid=GridSearchCV(estimator=pipe,
                    param_grid=param_grid,
                    cv=5,
                    n_jobs=-2,
                    verbose=verbosity,
                    scoring=['accuracy','precision'],
                    refit='precision')
    grid.fit(X_train,y_train)
    return grid


## Section 2: Logistic Regression
Logistic Regression models have many hyperparameters. We will focus on:
* `penalty` is a regularization parameter.
* `solver` specifies the type of algorithm used.
* `C` controls the strength of the penalty.

Not all penalties work for each solver.

These will be our initial choice for hyperparameters. They will help us narrow down our search and find other ranges to test.

In [6]:
thresholds = [round(0.1*i,2) for i in range(5,11)]
C = [10**(2*i+1) for i in range(-2,2)]
solver = ['newton-cg', 'newton-cholesky', 'lbfgs', 'liblinear', 'sag', 'saga']
penalty = ['none', 'l1', 'l2', 'elasticnet']
param_grid = [
            {'C':C,
             'solver':['lbfgs','newton-cg','newton-cholesky','sag'],
             'penalty':['l2',None]},
            {'C':C,
             'solver':['liblinear'],
             'penalty':['l1','l2']},
            {'C':C,
             'solver':['saga'],
             'penalty':['l1','l2',None,'elasticnet']}
              ]
logistic_param_grid = [{'model__'+key:value 
                                for key,value in param_dict.items()}
                                for param_dict in param_grid]
for params in logistic_param_grid:
    params['corr_selector__threshold']=thresholds


Now we are ready to do the grid search. We expect this to go well since Logistic Regression was our best performing model without any tuning. This initial training will help us establish a range to further tune the hyperparamters in.

In [7]:
from src.utils import save_df, get_df


def get_grid_results_df(pipe, name, dir, param_grid={}, verbosity=2):
    try:
        results_df = get_df(name, dir)
    except FileNotFoundError:
        pipe_grid_search = grid_search(
            X_TrainSet, Y_TrainSet, pipe, param_grid=param_grid, verbosity=verbosity
        )
        results_df = pd.DataFrame(pipe_grid_search.cv_results_)
        save_df(results_df, name, dir)
        # this normalizes the types
        results_df = get_df(name, dir)
    return results_df


logistic_pipe = create_pipe("LogisticRegression")
results_name = "logistic_grid_results_v1"
dir = "experiment_results/tuning/grids"

logistic_results_df_v1 = get_grid_results_df(
    logistic_pipe, results_name, dir, param_grid=logistic_param_grid
)


Let's look at what the best choices are at this stage.

In [8]:
from src.model_eval import get_best_params_df

get_best_params_df(logistic_results_df_v1)

Best parameters for current model:
threshold: 0.8
C: 0.001
penalty: None
solver: lbfgs
Avg. Precision: 88.70480236491775%.
Avg. Accuracy: 87.18030112459309%.

threshold: 0.8
C: 0.001
penalty: None
solver: newton-cg
Avg. Precision: 88.70480236491775%.
Avg. Accuracy: 87.18030112459309%.

threshold: 0.8
C: 0.001
penalty: None
solver: newton-cholesky
Avg. Precision: 88.70480236491775%.
Avg. Accuracy: 87.18030112459309%.



Clearly, the best correlation threshold is 0.8. The models have the same scores. Lets collect all of the parameters that have the same scores. We order the scores not by count but by score.

In [9]:
from src.model_eval import present_score_counts, score_stats


present_score_counts(logistic_results_df_v1)
best_score = score_stats(logistic_results_df_v1)


---  Score Counts  ---
Precision: 0.8870480236491776, Accuracy: 0.8718030112459308
Count: 34

Precision: 0.887042467962182, Accuracy: 0.8717743374108053
Count: 2

Precision: 0.8870152603635608, Accuracy: 0.8718030194666403
Count: 1

Precision: 0.8869924718004393, Accuracy: 0.8718890245305975
Count: 1

---  Score Stats  ---
Most Common: Precision: -1
             Accuracy: -1
             Count: 76
Max Score: Precision: 0.8870480236491776
           Accuracy: 0.8718030112459308
           Count: 34
Max Precision: 0.8870480236491776
Max Accuracy: 0.871917694255368


The most common scores are `nan` (we changed the score to be -1 if a `nan` value was showing up). This happens when a choice of parameters does not work well together. We won't worry about this as we are able to get quite good precision and accuracy with this first pass. Note that the best accuracy score is not far from the accuracy of the model with the best precision.

34 different choices of parameters had the best performance. We would like to see what these estimators had in common and look at neighborhoods around these parameters to see if we can improve the performance before moving on to the next model.

In [10]:
from src.model_eval import present_param_counts

present_param_counts(logistic_results_df_v1, best_score)

model__solver: saga, Count: 8
model__solver: lbfgs, Count: 6
model__solver: newton-cg, Count: 6
model__solver: newton-cholesky, Count: 6
model__solver: sag, Count: 5
model__solver: liblinear, Count: 3
model__penalty: None, Count: 20
model__penalty: l2, Count: 10
model__penalty: l1, Count: 4
model__C: 1000, Count: 13
model__C: 10, Count: 11
model__C: 0.001, Count: 5
model__C: 0.1, Count: 5
corr_selector__threshold: 0.8, Count: 34


We will make the following modifications to `logistic_param_grid`:
* remove `'liblinear'` and `'sag'` as they were used the least,
* pick a neighborhood around 0.8 for correlation threshold,
* focus on penalties `'l2'` and `'None'`
* focus on the range 1 to 1000 for `C`

We will see if focusing gives us any improvement in score.

In [11]:
from src.utils import divide_range

thresholds = divide_range(0.75,0.85,5)
C = divide_range(1,1000,6)
logistic_param_grid_v2=[{'model__C': C,
  'model__solver': ['lbfgs', 'newton-cg', 'newton-cholesky','saga'],
  'model__penalty': ['l2', None],
  'corr_selector__threshold': thresholds}]

In [12]:
logistic_results_df_v2 = get_grid_results_df(logistic_pipe,
                                             'logistic_results_df_v2', dir, 
                                             param_grid=logistic_param_grid_v2,
                                             verbosity=3)

Let's proceed by doing the analysis we did above of the results of this grid search.

In [13]:
get_best_params_df(logistic_results_df_v2)

Best parameters for current model:
threshold: 0.77
C: 1.0
penalty: None
solver: lbfgs
Avg. Precision: 88.70480236491775%.
Avg. Accuracy: 87.18030112459309%.

threshold: 0.77
C: 1.0
penalty: None
solver: newton-cg
Avg. Precision: 88.70480236491775%.
Avg. Accuracy: 87.18030112459309%.

threshold: 0.77
C: 1.0
penalty: None
solver: newton-cholesky
Avg. Precision: 88.70480236491775%.
Avg. Accuracy: 87.18030112459309%.



Again, very similar scores. So let's analyze the scores that showed up as we did before.

In [14]:
present_score_counts(logistic_results_df_v2)
best_score = score_stats(logistic_results_df_v2)

---  Score Counts  ---
Precision: 0.8870480236491776, Accuracy: 0.8718030112459308
Count: 156

Precision: 0.8868889708447109, Accuracy: 0.871716997961264
Count: 3

Precision: 0.8868481163465816, Accuracy: 0.8716883282364934
Count: 9

Precision: 0.8753149006289227, Accuracy: 0.8584414808786294
Count: 78

---  Score Stats  ---
Most Common: Precision: 0.8870480236491776
             Accuracy: 0.8718030112459308
             Count: 156
Max Score: Precision: 0.8870480236491776
           Accuracy: 0.8718030112459308
           Count: 156
Max Precision: 0.8870480236491776
Max Accuracy: 0.8718030112459308


Our most common score is our best score. It seems like we have chosen a good range of parameters since many of the combinations yield good results.

In [15]:
present_param_counts(logistic_results_df_v2, best_score)

model__solver: lbfgs, Count: 39
model__solver: newton-cg, Count: 39
model__solver: newton-cholesky, Count: 39
model__solver: saga, Count: 39
model__penalty: None, Count: 84
model__penalty: l2, Count: 72
model__C: 167.5, Count: 24
model__C: 334.0, Count: 24
model__C: 500.5, Count: 24
model__C: 667.0, Count: 24
model__C: 833.5, Count: 24
model__C: 1000.0, Count: 24
model__C: 1.0, Count: 12
corr_selector__threshold: 0.77, Count: 52
corr_selector__threshold: 0.79, Count: 52
corr_selector__threshold: 0.8099999999999999, Count: 52


All of the choices of parameters seem to be performing equally well, with insignificant exceptions. We will have to look at other metrics to determine distinguish between these choices of parameters. Things such as training time statistics, and standard deviation of the scores.

In [16]:
best_results = logistic_results_df_v2.query(f'mean_test_precision == {best_score[0]} and mean_test_accuracy == {best_score[1]}')
std_score_counts = {}

for _, row in best_results.iterrows():
    std_score = (row['std_test_precision'], row['std_test_accuracy'])
    if std_score in std_score_counts:
        std_score_counts[std_score] += 1
    else:
        std_score_counts[std_score] = 1
for key, value in std_score_counts.items():
    print(f"std_test_precision: {key[0]}"
          f"\nstd_test_accuracy: {key[1]}"
          f"\ncount: {value}")

std_test_precision: 0.0054844417109899
std_test_accuracy: 0.004118208175772
count: 156


So standard deviation will not help us distinguish either. This is annoying, but good. We will see how the models perform on the test data set.

In [17]:
import ast

def parameter_dicts(results_df, best_score):
    relevant = results_df.query(f'mean_test_precision == {best_score[0]} and mean_test_accuracy == {best_score[1]}')
    param_dicts = [ast.literal_eval(param_dict) for param_dict in relevant['params'].values]
    return param_dicts

best_params = parameter_dicts(logistic_results_df_v2, best_score)

best_pipes = []
for param_dict in best_params:
    base_pipe = create_pipe('LogisticRegression')
    pipe = base_pipe.set_params(**param_dict)
    pipe.param_dict = param_dict
    best_pipes.append(pipe)

model_params = {key.split('__')[0]:key.split('__')[1]
                for key in best_params[0].keys()}
count = 0
for pipe in best_pipes:
    print(f"Pipe {count}:")
    for step in pipe.get_params()['steps']:
        if step[0] in model_params:
            param = model_params[step[0]]
            value = step[1].get_params()[param]
            print(f"{step[0]}"
                  f"\n{param}: {value}")
    print()
    count += 1
    if count>=2:
        break


Pipe 0:
corr_selector
threshold: 0.77
model
solver: lbfgs

Pipe 1:
corr_selector
threshold: 0.77
model
solver: newton-cg



We will now train all of the above pipelines and evaluate them on the test dataset.

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [19]:
'''count = 0
for pipe in best_pipes:
    print(f"Training pipe {count}:")
    print(pipe.param_dict)
    pipe.fit(X_TrainSet, Y_TrainSet)
    count+=1
    '''

'count = 0\nfor pipe in best_pipes:\n    print(f"Training pipe {count}:")\n    print(pipe.param_dict)\n    pipe.fit(X_TrainSet, Y_TrainSet)\n    count+=1\n    '

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

'''
def evaluate_param_on_test_set(pipe,X_test, Y_test):
    y_pred = pipe.predict(X_test)
    accuracy = accuracy_score(Y_test, y_pred)
    precision = precision_score(Y_test, y_pred)
    recall = recall_score(Y_test, y_pred)
    f1 = f1_score(Y_test, y_pred)
    return ((precision, accuracy, recall, f1), pipe.param_dict)


def evaluate_and_sort(fitted_pipes, X_test, Y_test):
    evaluations = [evaluate_param_on_test_set(pipe, X_test, Y_test)
               for pipe in fitted_pipes]
    evaluation_dict = {}
    for eval in evaluations:
        if eval[0] in evaluation_dict:
            evaluation_dict[eval[0]].append(eval[1])
        else:
            evaluation_dict[eval[0]] = [eval[1]]
    sorted_eval_dict = {k:v for k,v in sorted(evaluation_dict.items(),
                                         key=lambda item: item[0],
                                         reverse=True)}
    return sorted_eval_dict

sorted_evals = evaluate_and_sort(best_pipes, X_TestSet, Y_TestSet)
print(len(sorted_evals))
'''

'\ndef evaluate_param_on_test_set(pipe,X_test, Y_test):\n    y_pred = pipe.predict(X_test)\n    accuracy = accuracy_score(Y_test, y_pred)\n    precision = precision_score(Y_test, y_pred)\n    recall = recall_score(Y_test, y_pred)\n    f1 = f1_score(Y_test, y_pred)\n    return ((precision, accuracy, recall, f1), pipe.param_dict)\n\n\ndef evaluate_and_sort(fitted_pipes, X_test, Y_test):\n    evaluations = [evaluate_param_on_test_set(pipe, X_test, Y_test)\n               for pipe in fitted_pipes]\n    evaluation_dict = {}\n    for eval in evaluations:\n        if eval[0] in evaluation_dict:\n            evaluation_dict[eval[0]].append(eval[1])\n        else:\n            evaluation_dict[eval[0]] = [eval[1]]\n    sorted_eval_dict = {k:v for k,v in sorted(evaluation_dict.items(),\n                                         key=lambda item: item[0],\n                                         reverse=True)}\n    return sorted_eval_dict\n\nsorted_evals = evaluate_and_sort(best_pipes, X_TestSet, Y

It turns out that all of these best sets of parameters produce models that perform equally well with respect to the standard meterics. We will have to pick one to deploy. We will look at the time it took to score each model during the grid search.

In [21]:
time_results = best_results.filter(['mean_score_time','std_score_time','params'])

time_results = time_results.sort_values(by=['mean_score_time','std_score_time'])
print(time_results.head())
params_choice = time_results.iloc[0]['params']


     mean_score_time  std_score_time  \
81          0.322713        0.144490   
108         0.377752        0.131118   
135         0.378142        0.145327   
65          0.383024        0.131624   
184         0.399108        0.108866   

                                                params  
81   {'corr_selector__threshold': 0.77, 'model__C':...  
108  {'corr_selector__threshold': 0.77, 'model__C':...  
135  {'corr_selector__threshold': 0.79, 'model__C':...  
65   {'corr_selector__threshold': 0.77, 'model__C':...  
184  {'corr_selector__threshold': 0.809999999999999...  


Let's look at the classification report for this model.

### Section 3: AdaBoost
We will now do a grid search with AdaBoost. After an initial search to determine a range, we will investigate more closely.

With AdaBoost, there are two types of parameters. We have parameters for AdaBoost and parameters for the weak learner it is using as a base estimator. The default base estimator is a Decision tree.

AdaBoost has the following hyperparameters.
* `n_estimators` the max number of estimators
* `learning_rate` which weights the estimators
* `algorithm` of which there are two choices

Decision trees classifiers have the following hyperparameters.
* `max_depth`
* `min_samples_split`
* `min_leaf_split`

In order to get some early results, we will break the hyperparameter grid into smaller pieces along certain axes. We will first investigate how the hyperparameters of AdaBoost impact the models and then we will tune the hyperparameters of the inner Decision tree.

In [22]:
thresholds = divide_range(0.55,0.95,5)

base_ada_params = [{'corr_selector__threshold': thresholds,
'model__n_estimators':[int(i) for i in divide_range(20,50,5)],
'model__learning_rate': divide_range(0.5,2,5),
'model__algorithm':['SAMME', 'SAMME.R']
}]


In [23]:
ada_pipe = create_pipe('AdaBoost')
base_ada_results_df_v1 = get_grid_results_df(ada_pipe,
                                             'base_ada_results_df_v1', dir, 
                                             param_grid=base_ada_params,
                                             verbosity=3)


Let's see what the top scores are.

In [24]:
present_score_counts(base_ada_results_df_v1)
best_score = score_stats(base_ada_results_df_v1)

---  Score Counts  ---
Precision: 0.8737315821516992, Accuracy: 0.8571224639110847
Count: 1

Precision: 0.8715598612273361, Accuracy: 0.8505849610338364
Count: 1

Precision: 0.871110904500443, Accuracy: 0.8542265339844135
Count: 1

Precision: 0.8702183448458646, Accuracy: 0.8482624954786097
Count: 1

---  Score Stats  ---
Most Common: Precision: -1
             Accuracy: -1
             Count: 72
Max Score: Precision: 0.8737315821516992
           Accuracy: 0.8571224639110847
           Count: 1
Max Precision: 0.8737315821516992
Max Accuracy: 0.8572946220117721


Let's look at the parameters associated with the top 5 scores.

In [25]:
from src.model_eval import collect_like_estimators

def top_n(results_df,num=5):
    estimators_by_score = collect_like_estimators(results_df)
    scores = list(estimators_by_score.keys())
    top = sorted(scores,reverse=True)[:num]
    for score in top:
        print(f"Score: {score}")
        present_param_counts(results_df, score)

top_n(base_ada_results_df_v1)

Score: (0.8737315821516992, 0.8571224639110847)
model__n_estimators: 50, Count: 1
model__learning_rate: 1.1, Count: 1
model__algorithm: SAMME.R, Count: 1
corr_selector__threshold: 0.79, Count: 1
Score: (0.8715598612273361, 0.8505849610338364)


model__n_estimators: 50, Count: 1
model__learning_rate: 1.4, Count: 1
model__algorithm: SAMME.R, Count: 1
corr_selector__threshold: 0.79, Count: 1
Score: (0.871110904500443, 0.8542265339844135)
model__n_estimators: 44, Count: 1
model__learning_rate: 1.1, Count: 1
model__algorithm: SAMME.R, Count: 1
corr_selector__threshold: 0.79, Count: 1
Score: (0.8702183448458646, 0.8482624954786097)
model__n_estimators: 44, Count: 1
model__learning_rate: 1.4, Count: 1
model__algorithm: SAMME.R, Count: 1
corr_selector__threshold: 0.79, Count: 1
Score: (0.8697431824167033, 0.8511012133767387)
model__n_estimators: 50, Count: 2
model__learning_rate: 1.1, Count: 2
model__algorithm: SAMME.R, Count: 2
corr_selector__threshold: 0.87, Count: 1
corr_selector__threshold: 0.95, Count: 1


This gives us an idea of how to narrow down our hyperparameters.
* We will focus on a range estimators from 42-53
* We will use the algorithm `'SAMME.R'`.
* We will focus on a range of learning rate of 0.9 to 1.6
* We will focus on a range of correlation threshold from 0.77 to 0.95

In [26]:
thresholds = divide_range(0.77,0.95,5)

base_ada_params_v2 = [{'corr_selector__threshold': thresholds,
'model__n_estimators':[int(i) for i in divide_range(42,53,5)],
'model__learning_rate': divide_range(0.9,1.6,5),
'model__algorithm':['SAMME.R']
}]

In [27]:
base_ada_results_df_v2 = get_grid_results_df(ada_pipe,'base_ada_results_df_v2',
                                             dir, param_grid=base_ada_params_v2,
                                             verbosity=3)

Let's see what the top scores are.

In [28]:
present_score_counts(base_ada_results_df_v2)
best_score = score_stats(base_ada_results_df_v2)

---  Score Counts  ---
Precision: 0.8740871643980797, Accuracy: 0.8533089630396896
Count: 2

Precision: 0.8740636696351014, Accuracy: 0.8561476562756898
Count: 2

Precision: 0.8733945549863253, Accuracy: 0.8556888338101345
Count: 2

Precision: 0.873384809314483, Accuracy: 0.8532515742658907
Count: 2

---  Score Stats  ---
Most Common: Precision: 0.8603154402588608
             Accuracy: 0.8471729431784553
             Count: 4
Max Score: Precision: 0.8740871643980797
           Accuracy: 0.8533089630396896
           Count: 2
Max Precision: 0.8740871643980797
Max Accuracy: 0.8570938928348294


Let's look at the parameters associated with the top 5 scores.

In [29]:
top_n(base_ada_results_df_v2)

Score: (0.8740871643980797, 0.8533089630396896)
model__n_estimators: 48, Count: 2
model__learning_rate: 1.32, Count: 2
model__algorithm: SAMME.R, Count: 2
corr_selector__threshold: 0.77, Count: 1
corr_selector__threshold: 0.806, Count: 1
Score: (0.8740636696351014, 0.8561476562756898)
model__n_estimators: 53, Count: 2
model__learning_rate: 1.18, Count: 2
model__algorithm: SAMME.R, Count: 2
corr_selector__threshold: 0.77, Count: 1
corr_selector__threshold: 0.806, Count: 1
Score: (0.8733945549863253, 0.8556888338101345)
model__n_estimators: 50, Count: 2
model__learning_rate: 1.18, Count: 2
model__algorithm: SAMME.R, Count: 2
corr_selector__threshold: 0.77, Count: 1
corr_selector__threshold: 0.806, Count: 1
Score: (0.873384809314483, 0.8532515742658907)
model__n_estimators: 50, Count: 2
model__learning_rate: 1.32, Count: 2
model__algorithm: SAMME.R, Count: 2
corr_selector__threshold: 0.77, Count: 1
corr_selector__threshold: 0.806, Count: 1
Score: (0.8733793850182583, 0.8531081639867153)
m

We will take the average of some of the above and then seaerch again looking at only the number of estimators and the learning rate. We feel we didn't cast a wide enough net with respect to these parameters.

In [30]:
base_ada_params_v3 = [{'corr_selector__threshold': [0.785],
'model__n_estimators':[int(i) for i in divide_range(40,70,5)],
'model__learning_rate': divide_range(1.2,5,5),
'model__algorithm':['SAMME.R']}]

base_ada_results_df_v3 = get_grid_results_df(ada_pipe,'base_ada_results_df_v3',
                                             dir, param_grid=base_ada_params_v3,
                                             verbosity=3)

In [31]:
present_score_counts(base_ada_results_df_v3)
best_score = score_stats(base_ada_results_df_v3)

---  Score Counts  ---
Precision: 0.8782214616422259, Accuracy: 0.8600471868731709
Count: 1

Precision: 0.8774926518476276, Accuracy: 0.8592443030482391
Count: 1

Precision: 0.8754282254137925, Accuracy: 0.8573519409095394
Count: 1

Precision: 0.8737115168166728, Accuracy: 0.8554595047844529
Count: 1

---  Score Stats  ---
Most Common: Precision: 0.7236725844374918
             Accuracy: 0.6599096050771103
             Count: 23
Max Score: Precision: 0.8782214616422259
           Accuracy: 0.8600471868731709
           Count: 1
Max Precision: 0.8782214616422259
Max Accuracy: 0.8600471868731709


The scores did improve, but not significantly. If the improvement is due to values for learning rate and estimators being outside of our earlier ranges, then we will have to continue searching.

In [32]:
top_n(base_ada_results_df_v3)

Score: (0.8782214616422259, 0.8600471868731709)
model__n_estimators: 70, Count: 1
model__learning_rate: 1.2, Count: 1
model__algorithm: SAMME.R, Count: 1
corr_selector__threshold: 0.785, Count: 1
Score: (0.8774926518476276, 0.8592443030482391)
model__n_estimators: 64, Count: 1
model__learning_rate: 1.2, Count: 1
model__algorithm: SAMME.R, Count: 1
corr_selector__threshold: 0.785, Count: 1
Score: (0.8754282254137925, 0.8573519409095394)
model__n_estimators: 58, Count: 1
model__learning_rate: 1.2, Count: 1
model__algorithm: SAMME.R, Count: 1
corr_selector__threshold: 0.785, Count: 1
Score: (0.8737115168166728, 0.8554595047844529)
model__n_estimators: 52, Count: 1
model__learning_rate: 1.2, Count: 1
model__algorithm: SAMME.R, Count: 1
corr_selector__threshold: 0.785, Count: 1
Score: (0.8720308867480462, 0.8536817598895137)
model__n_estimators: 46, Count: 1
model__learning_rate: 1.2, Count: 1
model__algorithm: SAMME.R, Count: 1
corr_selector__threshold: 0.785, Count: 1


So thee number of estimators jumped a fair bit. In the case of the Logistic Regression model, we got many many high performing sets of parameters. This made it difficult to focus on a few small neighborhoods (in the parameter space) of high performing sets of parameters. Our learning rate has stabilized, but we see that we should be casting a wider net.

In [33]:
base_ada_params_v4 = [{'corr_selector__threshold': [0.785],
'model__n_estimators':[int(i) for i in divide_range(40,90,10)],
'model__learning_rate': divide_range(1.1,1.4,5),
'model__algorithm':['SAMME.R']}]

base_ada_results_df_v4 = get_grid_results_df(ada_pipe,'base_ada_results_df_v4',
                                             dir, param_grid=base_ada_params_v4,
                                             verbosity=3)

In [34]:
present_score_counts(base_ada_results_df_v4)
best_score = score_stats(base_ada_results_df_v4)

---  Score Counts  ---
Precision: 0.8806046253682138, Accuracy: 0.8635739863865048
Count: 1

Precision: 0.879820653213331, Accuracy: 0.8624557479201604
Count: 1

Precision: 0.8797249483916806, Accuracy: 0.8628284584525335
Count: 1

Precision: 0.879675169418023, Accuracy: 0.8623123910755977
Count: 1

---  Score Stats  ---
Most Common: Precision: 0.8692879046528116
             Accuracy: 0.8519900652724344
             Count: 1
Max Score: Precision: 0.8806046253682138
           Accuracy: 0.8635739863865048
           Count: 1
Max Precision: 0.8806046253682138
Max Accuracy: 0.8635739863865048


In [35]:
top_n(base_ada_results_df_v4)

Score: (0.8806046253682138, 0.8635739863865048)
model__n_estimators: 90, Count: 1
model__learning_rate: 1.16, Count: 1
model__algorithm: SAMME.R, Count: 1
corr_selector__threshold: 0.785, Count: 1
Score: (0.879820653213331, 0.8624557479201604)
model__n_estimators: 80, Count: 1
model__learning_rate: 1.16, Count: 1
model__algorithm: SAMME.R, Count: 1
corr_selector__threshold: 0.785, Count: 1
Score: (0.8797249483916806, 0.8628284584525335)
model__n_estimators: 85, Count: 1
model__learning_rate: 1.16, Count: 1
model__algorithm: SAMME.R, Count: 1
corr_selector__threshold: 0.785, Count: 1
Score: (0.879675169418023, 0.8623123910755977)
model__n_estimators: 70, Count: 1
model__learning_rate: 1.16, Count: 1
model__algorithm: SAMME.R, Count: 1
corr_selector__threshold: 0.785, Count: 1
Score: (0.8795258538414708, 0.8619969172338957)
model__n_estimators: 80, Count: 1
model__learning_rate: 1.1, Count: 1
model__algorithm: SAMME.R, Count: 1
corr_selector__threshold: 0.785, Count: 1


We have narrowed down a decent neighborhood for estimators and learning rate. We will now look at tuning the Decision tree hyperparameters.

In [36]:
ada_params = {'corr_selector__threshold':[0.785],
            'model__n_estimators': [int(i) for i in divide_range(80,100,5)],
            'model__learning_rate': divide_range(1.1,1.3,5), 
            'model__algorithm':['SAMME.R']}
depths = [int(i) for i in divide_range(5,30)]+[None]
dt_params = {'max_depth': depths,
              'min_samples_split': [int(i) for i in divide_range(2,10)],
              'min_samples_leaf': [int(i) for i in divide_range(1,5)]}
new_dt_params = {"model__base_estimator__"+key:value 
                 for key,value in dt_params.items()}

ada_params.update(new_dt_params)


In [37]:
ada_dt_results_df_v1 = get_grid_results_df(ada_pipe,'ada_dt_results_df_v1',
                                             dir, param_grid=ada_params,
                                             verbosity=3)

### Beginning grid search for AdaBoost ###
Fitting 5 folds for each of 5400 candidates, totalling 27000 fits


AttributeError: 'NoneType' object has no attribute 'set_params'

In order to get intermediate results, we will train the AdaBoost model with different base estimators separately.